In [56]:
import os
from dotenv import load_dotenv
import pg8000
import pandas as pd
import datetime as dt
import random as rnd

# Cargar variables de entorno desde .env
load_dotenv()

PASS = os.getenv("POSTGRES_PASSWORD")

# Datos de conexión a la base de datos
db_params = {
    'host': 'localhost',
    'database': 'training',
    'user': 'postgres',
    'password': PASS,
    'port': 5432  # Puerto predeterminado de PostgreSQL
}

In [57]:
# get All table names
try:
    conn = pg8000.connect(**db_params)
    cursor = conn.cursor()
    select = "SELECT table_name FROM information_schema.tables"
    where = "WHERE table_schema = 'public'"
    cursor.execute(f'{select} {where}')
    table_names = cursor.fetchall()

    if table_names:
        table_names = [t[0] for t in table_names]
        print(table_names)
    else:
        print('No tables found')
except pg8000.Error as e:
    print(f'Error pg8000 conn: {e}')

finally:
    if 'cursor' in locals() or conn: 
        cursor.close()
        conn.close()

['empresas', 'vacantes', 'postulantes', 'postulantes_habilidades', 'habilidades', 'postulaciones', 'requerimientos']


In [58]:
try:
    conn = pg8000.connect(**db_params)
    cursor = conn.cursor()

    dataframes = {}
    # name = 'empresas'
    # cursor.execute(f'select * from {name}')
    # print(cursor.description)
    for name in table_names:
        cursor.execute(f'select * from {name}')
        cols = [item[0] for item in cursor.description]
        rows = cursor.fetchall()
        dataframes[name] = pd.DataFrame(rows, columns=cols)

except pg8000.Error as e:
    print(f'Error while exec query: {e}')
finally:
    if 'cursor' in locals() or conn: 
        cursor.close()
        conn.close()
    

In [59]:
# BASIC QUERIES
empresas = dataframes["empresas"]
vacantes = dataframes["vacantes"]
postulantes = dataframes["postulantes"]
habilidades = ["habilidades"]
postulantes_habilidades = dataframes["postulantes_habilidades"]
postulaciones = dataframes["postulaciones"]
requerimientos = dataframes["requerimientos"]


In [60]:
# empresas del sector salud
# empresas[empresas['sector'] == 'Tecnología']
# vacantes publicadas despues o antes de una fecha dada
# dt.datetime.strptime('2024-1-1',"%Y-%m-%d")
# vacantes['fecha_publicacion']

# vacantes filtradas por fechas (año, mes o fecha completa)
# vacantes['fecha_publicacion'] = pd.to_datetime(vacantes['fecha_publicacion'])
# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'].dt.year < 2024]

# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'].dt.month == 10]

# filter_date = pd.to_datetime('2023-12-1')
# filter_date
# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'] > filter_date]
# vacantes_filtradas

In [61]:
# postulantes que sin email o con correos que no sean gmail
# es importante entender que primero va el filtro de NaN o None
# esto es asi porque si se usa str.contains() antes no va a manejarlos bien y
# va a ignorar el filtro el .isnull()
# postulantes
postulantes[((postulantes['email_postulante'].isnull()) | postulantes['email_postulante'].str.contains("gmail"))]

,id_postulante,nombre_postulante,apellido_postulante,email_postulante
4,4,Pedro,Ruiz,pedro@gmail.com
5,6,Roberto,Santos,None


In [62]:
# inserts de empresas
# nueva_empresa = {'id_empresa': len(empresas['id_empresa'])+1,'nombre_empresa': 'HTec', 'sector': 'Salud'}
# empresas = pd.concat([empresas, pd.DataFrame([nueva_empresa])], ignore_index=True)

# otra forma de hacerlo es usando loc
# nueva_empresa = {'id_empresa': len(empresas['id_empresa'])+1,'nombre_empresa': 'ITS', 'sector': 'Tecnología'}
# empresas.loc[len(empresas)] = nueva_empresa
# empresas

In [63]:
# Actualizar datos una empresa
# renombrarla
# empresas.loc[empresas['nombre_empresa'] == 'HTec', 'sector'] = 'Farmaceutico'
# empresas.loc[empresas['nombre_empresa'] == 'Nueva Empresa', 'sector'] = 'Tecnología'
# empresas

# eliminar por indice o por nombre
# empresas
# empresas[empresas['nombre_empresa'] != 'ITS']

In [64]:
# eliminar o encontrar una fila por indice
# indices = empresas[empresas['sector'] == 'Salud'].index
# empresas.iloc[indices]
# empresas.drop(indices)
empresas
# empresas[empresas['sector'] == 'Salud']

,id_empresa,nombre_empresa,sector
0,1,Tech Innovators Inc.,Tecnología
1,2,Global Finance Ltd.,Finanzas
2,3,Health Solutions SA,Salud


In [65]:
# JOINS
# vacantes con nombre de empresa
# empresas
# vacantes
# vxe = pd.merge(vacantes, empresas, left_on='id_empresa', right_on='id_empresa', how='inner')
# vxe = vxe.loc[:,['titulo_vacante', 'nombre_empresa']]
# vxe

In [66]:
# postulaciones con informacion del postulante (incluyendo postulantes sin postulacion)
# left join
# postulantes
# postulaciones
# kpxp = 'id_postulante'
# pxp = pd.merge(postulaciones, postulantes, how='left', left_on=kpxp, right_on=kpxp)
# pxp.iloc[:, 2::2]

In [67]:
# postulante, titulo de vacante y empresa, en 2024 y sin fecha de post 
k1 = 'id_postulante'
k2 = 'id_vacante'
k3 = 'id_empresa'
pxpv = pd.merge(postulaciones, postulantes, on=k1).merge(vacantes, on=k2).merge(empresas, on=k3)
pxpv.loc[:, ['nombre_postulante','titulo_vacante','nombre_empresa']]

,nombre_postulante,titulo_vacante,nombre_empresa
0,Laura,Desarrollador Full Stack,Tech Innovators Inc.
1,Carlos,Analista Financiero Senior,Global Finance Ltd.
2,Ana,Desarrollador Full Stack,Tech Innovators Inc.
3,Ana,Data Scientist,Tech Innovators Inc.
4,Pedro,Oficinista entry level,Health Solutions SA
5,Sofía,Desarrollador Full Stack,Tech Innovators Inc.
6,Sofía,Desarrollador SQL,Tech Innovators Inc.
7,Ana,Analista Financiero Senior,Global Finance Ltd.
8,Pedro,Desarrollador SQL,Tech Innovators Inc.
9,Ana,Desarrollador Full Stack,Tech Innovators Inc.
